In [1]:
from sentence_transformers import SentenceTransformer

# Load the model
model = SentenceTransformer("Qwen/Qwen3-Embedding-8B")

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen/Qwen3-Embedding-8B/resolve/main/modules.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7b24fdf75fd0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: ef5777d1-1480-470e-8b2b-ef43701f5b02)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen3-Embedding-8B/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

In [1]:
import pandas as pd
import os
from torch.utils.data import DataLoader

In [2]:
def read_xml_files(folder_path):
    """Read all XML files in a folder and return a concatenated DataFrame."""
    # List all XML files in the folder
    xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]
    xml_files.sort()

    # Read each XML file and append to a list
    df_list = []
    for xml_file in xml_files:
        file_path = os.path.join(folder_path, xml_file)
        df = pd.read_xml(file_path)
        df_list.append(df)

    # Concatenate all DataFrames in the list
    concatenated_df = pd.concat(df_list, ignore_index=True)

    return concatenated_df

In [3]:
data = read_xml_files('/disk/scratch/s1891075/AIP_data/nls-catalogue-published-material/nls-catalogue-published-material_dc')


In [7]:
dataset = data.to_dict(orient='records', index=True)


In [6]:
dataset[0]

{'title': 'Travel /',
 'creator': 'Leeson, Edward,1947-2011.',
 'type': 'text',
 'publisher': 'London J. Murray',
 'date': '1980',
 'language': 'eng',
 'subject': 'Short stories, English.',
 'description': None,
 'coverage': None,
 'relation': None,
 'rights': None,
 'identifier': None,
 'format': None}

In [14]:
def collate_fn(batch):
    texts = []
    items = []
    for row in batch:
        texts.append(f"Titile: {row['title']}\nAuthors: {row['creator']}\nSubject: {row['subject']}\nRelation: {row['relation']}\nLangeuage: {row['language']}")
        items.append(batch)
    return {'texts': texts, 'items': items}

In [15]:
loader = DataLoader(dataset, batch_size=16, shuffle=False, drop_last=False, collate_fn=collate_fn)

In [16]:
batch = next(iter(loader))

In [19]:
batch

{'texts': ['Titile: Travel /\nAuthors: Leeson, Edward,1947-2011.\nSubject: Short stories, English.\nRelation: None\nLangeuage: eng',
  'Titile: Resource book of test items in chemistry\nAuthors: Jenkins, E. W.(Edgar William)\nSubject: Chemistry\nRelation: None\nLangeuage: eng',
  'Titile: Arbitration for contractors\nAuthors: Stephenson, Douglas A.\nSubject: None\nRelation: None\nLangeuage:    ',
  'Titile: Armorial bearings of the sovereigns of England a short account\nAuthors: Standing Conference for Local History.\nSubject: Heraldry\nRelation: None\nLangeuage: eng',
  "Titile: Sharing caring : caring, equal opportunities and the voluntary sector : a topic paper from NCVO's Community Care Project and CVSNA /\nAuthors: Thompson, Catherine.\nSubject: None\nRelation: None\nLangeuage:    ",
  'Titile: Environmental impact assessment a bibliography with abstracts\nAuthors: Wathern, Peter.\nSubject: Environmental impact analysis\nRelation: None\nLangeuage: eng',
  'Titile: Activities in th

In [34]:
ebd = model.encode(batch)

In [38]:
model.device

device(type='cuda', index=0)

In [39]:
ebd.shape

(16, 4096)

In [9]:
len(loader)

318215

In [21]:
data.type.unique()

array(['text', 'notated music', 'BroadsidesEnglandLondon1801-1900.rbgenr',
       ..., 'IncunabulaItalyVenice1493.',
       'Science fiction comic books, strips, etc.Japan.',
       'PoemsEnglish19th century.'], shape=(4100,), dtype=object)

In [22]:
1272857 * 4

5091428

In [20]:
data.head(100)


,title,creator,type,publisher,date,language,subject,description,coverage,relation,rights,identifier,format
0,Travel /,"Leeson, Edward,1947-2011.",text,London J. Murray,1980,eng,"Short stories, English.",None,None,None,None,None,None
1,Resource book of test items in chemistry,"Jenkins, E. W.(Edgar William)",text,London Murray,1981,eng,Chemistry,Bibliography: p15,None,None,None,None,None
2,Arbitration for contractors,"Stephenson, Douglas A.",text,Northwood Books,1982,,None,None,None,None,None,None,None
3,Armorial bearings of the sovereigns of England...,Standing Conference for Local History.,text,London Bedford Square Press [for the] Standing...,1977,eng,Heraldry,Bibliography: p.29-31,None,None,None,None,None
4,"Sharing caring : caring, equal opportunities a...","Thompson, Catherine.",text,Community Care Project,[1985],,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,MacTavish of Dunardry,"Bradford, E. F.(Ernest Frederick),1917-",text,Whitby E.F.Bradford,1991,eng,McTavish family.,Includes index,None,None,None,None,None
96,Kado's look and see.,None,text,"Loughborough : Balloon Books,",1991.,eng,None,Cover title.,None,None,None,None,None
97,Bathing water quality.,Great Britain.Department of the Environment.,text,"[London] : Department of the Environment,",[1991?].,eng,Water quality management,Cover title.,None,None,None,None,None
98,Enzymes in food processing /,"Woods, L. F. J.",text,"Glasgow : Blackie,",1991.,eng,Food industry and trade.,Includes bibliographical references (p. ) and ...,None,None,None,None,None


In [23]:
len(data)

5091427

In [15]:
data.language.describe()

count     5091057
unique        367
top           eng
freq      4171764
Name: language, dtype: object

In [ ]:
data.language.unique() 

array(['eng', 'ger', 'grc', 'wel', '   ', 'ita', 'fre', 'und', 'esk',
       'sho', 'ewe', 'rum', 'rus', 'dut', 'lat', 'pol', 'zxx', 'chi',
       'gre', 'spa', 'gla', 'ice', 'dan', 'cat', 'mul', 'iri', 'nor',
       'fri', 'ukr', 'map', 'swe', 'por', 'heb', 'fro', 'gle', 'slo',
       'sco', 'fin', 'ben', 'enm', 'cze', 'far', 'jpn', 'may', 'san',
       'ara', 'bul', 'afr', 'alb', 'hun', 'ine', 'ind', 'mao', 'per',
       'pli', 'bre', 'xho', 'tur', 'tha', 'ach', 'twi', '|||', 'snd',
       'slv', None, 'urd', 'ang', 'aze', 'kor', 'yid', 'kik', 'mal',
       'est', 'hrv', '|d', 'epo', 'mar', 'lit', 'efi', 'suk', 'pan',
       'swa', 'fao', 'cor', 'scc', 'nic', 'gaa', 'som', 'snh', 'sna',
       'guj', 'glg', 'vie', 'bel', 'geo', 'frm', 'elx', 'lan', 'wen',
       'him', 'bur', 'kon', 'lub', 'gmh', 'sso', 'got', 'sao', 'kab',
       'lap', 'esp', 'mlt', 'mol', 'sla', 'pus', 'dak', 'tag', 'nya',
       'chu', 'lav', 'mla', 'yor', 'roa', 'kok', 'mon', 'ssa', 'srp',
       'hin', 'nde', '

In [13]:
sample = data.sample(frac=0.05)

In [14]:
sample.to_csv('random_sample.csv')